In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# Import the CRUD module
from CRUD import AnimalShelter

# Authenticate to the AAC database
username = "aacuser"
password = "userPassword1"
shelter = AnimalShelter(username, password)


# Retrieve all data from the database 
df = pd.DataFrame.from_records(shelter.read_all({}))


app = JupyterDash('SimpleExample')

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Position logo, dashboard title, and unique identifier
app.layout = html.Div([
    # Add image with anchor to snhu.edu
    html.Center(html.A([html.Img(height='150', width='150', 
                        src='data:image/png;base64,{}'.format(encoded_image.decode()))],
                href='https://www.snhu.edu')),
    html.Center(html.B(html.H1('Grazioso Salvare Rescue Animals'))),
    html.H6("Dashboard Author: Alison Croteau"),
    html.Hr(),
    html.Div(
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ], 
        style_table={'overflowX':'scroll'},
        data=df.to_dict('records'),
        # Set up the features for the interactive data table to make it user-friendly for the client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable='multi',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,   
    ),
    html.Br(),
    html.Hr(),
])

    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
                
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns[1:]]
    data=df.to_dict('records')
            
    return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    """Update style"""
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]




app